In [1]:
pip install tensorflow


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib



In [3]:
# Load the motion detection dataset
df = pd.read_csv("data/pose_data_with_labels.csv")
# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform the 'category' column
df['label'] = le.fit_transform(df['label'])
df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,z24,z25,z26,z27,z28,z29,z30,z31,z32,label
0,0.656242,0.657945,0.658258,0.658370,0.650787,0.647205,0.643332,0.641253,0.621745,0.646159,...,-0.136605,0.060308,-0.257929,0.248826,-0.112656,0.260050,-0.103744,0.160585,-0.247247,3
1,0.498194,0.505137,0.509107,0.512814,0.492173,0.487455,0.483065,0.517913,0.481324,0.505514,...,-0.011005,-0.137239,-0.130165,0.056535,0.051961,0.066913,0.064734,-0.096526,-0.090435,3
2,0.503693,0.510164,0.514853,0.519486,0.498114,0.494252,0.490576,0.527839,0.486349,0.512618,...,-0.007757,-0.222389,-0.409163,0.005740,-0.019116,0.023248,-0.011919,-0.100287,-0.166244,3
3,0.663763,0.671197,0.676193,0.680849,0.658865,0.654813,0.651177,0.691068,0.651258,0.673122,...,0.014669,-0.086695,-0.081398,0.075016,0.073768,0.083189,0.082889,-0.054126,-0.055695,3
4,0.531094,0.524614,0.521559,0.518423,0.532225,0.534428,0.536833,0.509513,0.533771,0.525086,...,-0.053331,0.247773,-0.035642,0.282698,-0.069808,0.284957,-0.082939,0.281320,-0.125497,3


In [6]:
X = df.drop('label', axis=1)
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Now use fit_transform correctly
X_test_scaled = scaler.transform(X_test)

# Build a deep neaural network model for  motion detection

In [8]:
nn_model = Sequential([
    Dense(128, input_dim=X_train_scaled.shape[1],activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
#Compile the model
nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
nn_model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
244/244 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.3606 - loss: 2.2350 - val_accuracy: 0.5925 - val_loss: 1.3884
Epoch 2/20
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6251 - loss: 1.2754 - val_accuracy: 0.6653 - val_loss: 1.1565
Epoch 3/20
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6921 - loss: 1.0715 - val_accuracy: 0.6981 - val_loss: 1.0391
Epoch 4/20
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7335 - loss: 0.9124 - val_accuracy: 0.7068 - val_loss: 0.9809
Epoch 5/20
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7585 - loss: 0.8572 - val_accuracy: 0.7294 - val_loss: 0.9275
Epoch 6/20
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7621 - loss: 0.7929 - val_accuracy: 0.7458 - val_loss: 0.8837
Epoch 7/20
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7705 - loss: 0.7758 - val_accuracy: 0.7591 - val_loss: 0.8681
Epoch 8/20
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7881 - loss: 0.7129 - val_accuracy: 0.

In [10]:
# evaluate the model
y_pred_nn = nn_model.predict(X_test_scaled)
y_pred_nn_classes = np.argmax(y_pred_nn, axis=1)
print(f"Neural Netowrk Accuracy: {accuracy_score(y_test, y_pred_nn_classes)}")

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Neural Netowrk Accuracy: 0.7757277572775728


## Ensembel Model (RandomForest + Neural Network_ for Motion Detection

In [11]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [14]:
# Train a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [15]:
# Get predictions from both models (Random Forest + Neural Network)
rf_preds = rf_model.predict(X_test_scaled)
nn_preds = np.argmax(nn_model.predict(X_test_scaled), axis=1)

# Ensemble majroity voting
ensemble_preds = np.round((rf_preds + nn_preds) / 2)

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [17]:
# Evaluate the ensemble model
print(f"Ensemble Model Accuracy: {accuracy_score(y_test, ensemble_preds)}")
print("Classification Report:\n", classification_report(y_test, ensemble_preds))

Ensemble Model Accuracy: 0.7224272242722427
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.60      0.68       121
           1       0.71      0.55      0.62        87
           2       0.59      0.62      0.60        84
           3       0.93      0.53      0.67        97
           4       0.44      0.72      0.55        60
           5       0.76      0.47      0.58       101
           6       0.55      0.83      0.66        87
           7       0.81      0.69      0.74       135
           8       0.67      0.81      0.73       118
           9       0.81      0.74      0.77       185
          10       0.59      0.89      0.71       105
          11       0.71      0.81      0.76        97
          12       0.73      0.93      0.82       167
          13       0.72      0.65      0.68       126
          14       0.65      0.76      0.70       103
          15       0.78      0.83      0.80        64
          16 

In [19]:
# Save the trained model to a file
joblib.dump(rf_model, 'random_forest_model.pkl')
print("Model saved successfully!")

Model saved successfully!


In [20]:
joblib.dump(nn_model, 'neural_network.pkl')
print("Model saved successfully!")

Model saved successfully!
